## PASO 1: Importar librerías y configurar el dispositivo

In [1]:
import torch
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.animation import FuncAnimation
import os

In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [3]:
# Parámetros
nx = ny = 50   # Número de puntos de malla en x e y
n_hidden = 20  # Tamaño de la capa oculta
n_layers = 3   # Número de capas ocultas
n_epochs = 10000  # Número de épocas de entrenamiento
batch_size = 100   # Tamaño del mini-lote

## PASO 2: Definir la condición inicial y la solución exacta

In [4]:
# # Definir la condición inicial gaussiana
# def gaussian(x, y, x0, y0, sigma):
#     return np.exp(-((x - x0)**2 + (y - y0)**2)/(2*sigma**2))
def gaussian(x, y, x0, y0, sigma):
    return torch.exp(-((x - x0)**2 + (y - y0)**2)/(2*sigma**2))

In [5]:
x = np.linspace(0, 1, nx)
y = np.linspace(0, 1, ny)
X, Y = np.meshgrid(x, y)

In [6]:
# Parámetros de la condición inicial gaussiana
x0, y0 = 0.5, 0.5
sigma = 0.1

# u_init = gaussian(X, Y, x0, y0, sigma)

In [7]:
# Definir la solución exacta de la ecuación de calor en dos dimensiones
def exact_solution(x, y, t):
    # return torch.exp(-2*t)*gaussian(x, y, x0, y0, np.sqrt(sigma**2 + 4*t))
    return torch.from_numpy(np.exp(-2*t))*gaussian(x, y, x0, y0, np.sqrt(sigma**2 + 4*t))

## PASO 3: Definir la ecuación de calor y las funciones de pérdida

In [8]:
# Definir la ecuación de calor en dos dimensiones
def f(x, y, t, u):
    # Derivadas parciales de u
    ux = torch.autograd.grad(u, x, torch.ones_like(u), create_graph=True)[0]
    uxx = torch.autograd.grad(ux, x, torch.ones_like(u), create_graph=True)[0]
    uy = torch.autograd.grad(u, y, torch.ones_like(u), create_graph=True)[0]
    uyy = torch.autograd.grad(u, y, torch.ones_like(u), create_graph=True)[0]
    
    # Definir la función de pérdida de la ecuación de calor
    u_t = torch.autograd.grad(u.sum(), t, create_graph=True)[0]
    # Ecuación de calor en dos dimensiones
    return u_t - 0.01*(uxx + uyy)

In [9]:
# Definir la función de pérdida de la ecuación de calor
def loss_heat(u, x, y, t):
    u_t = torch.ones_like(u)*t
    f_u = f(x, y, t, u)
    return torch.mean((u_t - f_u)**2)

In [10]:
# Definir la función de pérdida de las condiciones iniciales
def loss_init(u, u_init):
    return torch.mean((u - u_init)**2)

In [11]:
# Definir la función de pérdida total
def loss_total(u, x, y, t, u_init):
    return loss_heat(u, x, y, t) + loss_init(u, u_init)

## PASO 4: Generar la malla y los datos de entrenamiento

In [12]:
# Generar la malla
X, Y = np.meshgrid(x, y)
X_tensor = torch.from_numpy(X).float().to(device)
Y_tensor = torch.from_numpy(Y).float().to(device)

In [13]:
# # Convertir la condición inicial a un tensor y enviarla a la GPU
# u_init_tensor = torch.from_numpy(u_init).float().to(device)
# u_init_tensor = u_init_tensor.requires_grad_(True)
u_init = gaussian(X_tensor, Y_tensor, x0, y0, sigma)
u_init_tensor = u_init.float().to(device).requires_grad_(True)

In [14]:
# def gaussian(x, y, x0, y0, sigma):
#     return torch.exp(-((x - x0)**2 + (y - y0)**2)/(2*sigma**2))

# u_init = gaussian(X, Y, x0, y0, sigma)
# u_init_tensor = torch.from_numpy(u_init).float().to(device)
# u_init_tensor = u_init_tensor.requires_grad_(True)
X_tensor = torch.from_numpy(X).float().to(device)
Y_tensor = torch.from_numpy(Y).float().to(device)
x0_tensor = torch.tensor(x0, dtype=torch.float32, device=device)
y0_tensor = torch.tensor(y0, dtype=torch.float32, device=device)
sigma_tensor = torch.tensor(sigma, dtype=torch.float32, device=device)

def gaussian(x, y, x0, y0, sigma):
    return torch.exp(-((x - x0)**2 + (y - y0)**2)/(2*sigma**2))

u_init = gaussian(X_tensor, Y_tensor, x0_tensor, y0_tensor, sigma_tensor)
u_init_tensor = u_init.requires_grad_(True)


In [15]:
# Crear los datos de entrenamiento
t_min, t_max = 0, 0.5
t = np.linspace(t_min, t_max, 101)
t_tensor = torch.from_numpy(t).float().to(device)

In [16]:
x_data = torch.cat([X_tensor.reshape(-1, 1), Y_tensor.reshape(-1, 1), torch.zeros(nx*ny, 1).to(device)], dim=1)
y_data = torch.zeros(nx*ny, len(t))
for i in range(len(t)):
    y_data[:, i] = torch.from_numpy(exact_solution(X, Y, t[i]).reshape(-1))

TypeError: expected np.ndarray (got numpy.float64)

In [ ]:
# Convertir los datos de entrenamiento a tensores y enviarlos a la GPU
x_data_tensor = x_data.float().to(device)
y_data_tensor = y_data.float().to(device)

In [ ]:
# Definir la red neuronal
class Net(torch.nn.Module):
    def __init__(self, n_hidden, n_layers):
        super(Net, self).__init__()
        self.layers = torch.nn.ModuleList()
        self.layers.append(torch.nn.Linear(3, n_hidden))
        for i in range(n_layers-1):
            self.layers.append(torch.nn.Linear(n_hidden, n_hidden))
        self.layers.append(torch.nn.Linear(n_hidden, 1))

    def forward(self, x):
        for i in range(len(self.layers)-1):
            x = torch.relu(self.layers[i](x))
        x = self.layers[-1](x)
        return x

In [ ]:
n_hidden = 100
n_layers = 4
net = Net(n_hidden, n_layers).to(device)

In [ ]:
optimizer = torch.optim.Adam(net.parameters(), lr=1e-3)
mse_loss = torch.nn.MSELoss()

In [ ]:
n_epochs = 10000
for epoch in range(n_epochs):
    optimizer.zero_grad()
    u_pred = net(x_data_tensor).requires_grad_(True)

    loss = loss_total(u_pred, X_tensor, Y_tensor, t_tensor, u_init_tensor)
    loss.backward()
    optimizer.step()
    if (epoch+1) % 1000 == 0:
        print('Epoch [{}/{}], Loss: {:.4e}'.format(epoch+1, n_epochs, loss.item()))